In [98]:
import math
import numpy as np
import pandas as pd

from avipy import qty, atmosphere as atm

# Landing Performance Assignment
## Deliberable 2: Actual Landing Distance

Calculate the actual landing distance for a landing weight of 250000 kg at sea level.

Values:
- Final approach speed, $ v_{ref} + 5 Kts $
- Flare time, $ t_f = 5.2 s $
- Flare speed ratio, $ FSR = 0.9763 $
- Transition time, $t_{tr} = 1.8 s$
- Transition speed decay, $decay = 0.982 $
- Thrust at touchdown per engine, $ T_{td} = 3400 lbs $

Assumptions:
- No reverse thrust, no slope, no wind
- 1 second to apply maximum braking power
- Ground idle reverse thrust as follows:
- Any flare height is negligible, thus  $TAS_{stall} = IAS_{stall} = 122 Kts$
- Stall speed to ref speed, $ v_{ref} = v_{stall} \cdot 1.23 $
- Stall speed to touchdown speed, $ v_{td} = v_{stall} \cdot 1.18 $


| TAS [kts] | Total Idle Forward Thrust [lbs] |
|-----------|---------------------------------|
| 130 | 4896 |
| 120 | 5266 |
| 110 | 5517 |
| 100 | 5949 |
| 80 | 6580 |
| 60 | 7235 |
| 40 | 8157 |
| 20 | 8991 |
| 0 | 9875 |

In [99]:
mass = qty.Mass(250000)
weight = qty.Force.Kg(mass)
v_stall = qty.Velocity.Kts(122)
v_stall_to_ref = 1.23
v_stall_to_td = 1.18
t_flare = qty.Time(5.2)
fsr = 0.9763
t_trans = qty.Time(1.8)
speed_decay = 0.982
thrust_td = qty.Force.Kg(qty.Mass.Lbs(3400 * 2))
mu_dry = 0.54
density = 1.2252
cl_gnd = 0.126
cd_gnd = 0.213
surface = qty.Area(436.8)

speed_thrust = [
    (qty.Velocity.Kts(130), qty.Force.Kg(qty.Mass.Lbs(4896))),
    (qty.Velocity.Kts(120), qty.Force.Kg(qty.Mass.Lbs(5266))),
    (qty.Velocity.Kts(110), qty.Force.Kg(qty.Mass.Lbs(5517))),
    (qty.Velocity.Kts(100), qty.Force.Kg(qty.Mass.Lbs(5949))),
    (qty.Velocity.Kts(80), qty.Force.Kg(qty.Mass.Lbs(6580))),
    (qty.Velocity.Kts(60), qty.Force.Kg(qty.Mass.Lbs(7235))),
    (qty.Velocity.Kts(40), qty.Force.Kg(qty.Mass.Lbs(8157))),
    (qty.Velocity.Kts(20), qty.Force.Kg(qty.Mass.Lbs(8991))),
    (qty.Velocity.Kts(0), qty.Force.Kg(qty.Mass.Lbs(9875))),
]

Calculate the flare distance, using:\
$ \large S_{air} = v_{flare_{avg}} \cdot t_{flare} $

In [100]:
v_ref = qty.Velocity(v_stall * v_stall_to_ref)
v_app = qty.Velocity(v_ref + qty.Velocity.Kts(5))
v_td = qty.Velocity(v_stall * v_stall_to_td)
v_bo = qty.Velocity(v_td * speed_decay)

print(v_app, v_ref, v_td, v_bo)

79.77 m/s 77.20 m/s 74.06 m/s 72.73 m/s


In [101]:
v_avg = qty.Velocity((v_app + v_app * fsr) / 2)
dist_air = qty.Distance(v_avg * t_flare)

print(dist_air)

409.89 meters


Now, calculate the transition distance. First, the average speed during the transition time can be obtained by taking the average between the touchdown speed and the brakes-on speed.

In [102]:
v_bo = qty.Velocity(v_td * speed_decay)
v_trans_avg = qty.Velocity((v_td + v_bo) / 2)

print(v_trans_avg)

73.39 m/s


Then the transition distance can be obtained by applying the transition speed and transition time.

In [103]:
dist_trans = qty.Distance(v_trans_avg * t_trans)

print(dist_trans)

132.11 meters


The ground run distance can be calculated with a multi-step practical approach:

1. Split the round run up into multiple speed segments. 
2. Calculate the acceleration at the upper speed and lower speed of the segment.
3. Calculate the average speed and acceleration during the segment
4. With the average speed and acceleration, calculate the distance travelled throughout the segment.
5. Add up all segment distances, the total is the approximate ground run distance.

To calculate this in Python, two functions will be defined. The first function `get_brake_dist` takes the brakes-on speed, and divides the ground run into multiple speed segments. For every segment, the second function will be called.

The second function `get_segment_dist` takes the speed bounds of the segment, and calculates the segment distance using the described approach. 

To calculate the acceleration at a given speed. The following equation is used:

$\large a = \frac{g \cdot (T - D - \mu \cdot (W - L))}{W}$

where

$L = \frac{1}{2} \cdot \rho_h \cdot v^2 \cdot S \cdot C_{L_{gnd}}$

and 

$D = \frac{1}{2} \cdot \rho_h \cdot v^2 \cdot S \cdot C_{D_{gnd}}$

In [104]:
def get_brake_dist(v_bo: qty.Velocity, segments: list[tuple[qty.Velocity, qty.Force]]) -> qty.Distance:
    brake_dist = 0
    
    for i in range(len(segments)):
        if i >= len(segments) - 1:
            return qty.Distance(brake_dist)

        segment = get_segment_dist(segments[i], segments[i + 1])
        brake_dist += segment
    
    return None
    
def get_segment_dist(upper: tuple[qty.Velocity, qty.Force], lower: tuple [qty.Velocity, qty.Force]) -> qty.Distance:
    speed_upper, speed_lower = upper[0], lower[0]
    thrust_upper, thrust_lower = upper[1], lower[1]
    
    drag_upper = qty.Force(0.5 * density * speed_upper**2 * surface * cd_gnd)
    drag_lower = qty.Force(0.5 * density * speed_lower**2 * surface * cd_gnd)
    
    lift_upper = qty.Force(0.5 * density * speed_upper**2 * surface * cl_gnd)
    lift_lower = qty.Force(0.5 * density * speed_lower**2 * surface * cl_gnd)

    accel_upper = (9.81 *  (thrust_upper - drag_upper - mu_dry * (weight - lift_upper))) / weight
    accel_lower = (9.81 *  (thrust_lower - drag_lower - mu_dry * (weight - lift_lower))) / weight

    accel_avg = (accel_upper + accel_lower) / 2
    v_avg = qty.Velocity((speed_upper + speed_lower) / 2)
    delta_v = qty.Velocity(speed_upper - speed_lower)
    segment_dist = qty.Distance(-(v_avg * delta_v / accel_avg))
    
    return qty.Distance(segment_dist)

Insert the speed and thrust at the brakes-on speed in the beginning of the speed_thrust list. Then obtain the distance using the function defined above.

In [105]:
speed_thrust.insert(0, (v_bo, thrust_td))

dist_brake = get_brake_dist(v_bo, speed_thrust)
print(dist_brake)

473.62 meters


In [106]:
dist_total = qty.Distance(dist_air + dist_trans + dist_brake)

print(dist_total)

1015.62 meters
